# Benzinga-Nachrichten-Verarbeitung

## Setup up

### Cluster spin up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
cwd="/content/drive/MyDrive/NewsTrading/trading_bot"
%cd /content/drive/MyDrive/NewsTrading/trading_bot
%pip install -r requirements_clean.txt

In [ ]:
import sys, io, tarfile, os
import re
import pandas as pd
import subprocess
import os

import dask
import dask.dataframe as dd
from dask.distributed import Client
from distributed.diagnostics.plugin import WorkerPlugin
from pyngrok import ngrok, conf

from src.preprocessing.news_parser import get_company_abbreviation, yahoo_get_wrapper, \
                                          infer_author, filter_body

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

In [ ]:
## Set Up NGROK-Tunnel
conf.get_default().auth_token = "2WntwErWDt9LxQ2Jfp6C8OxDAMK_7iZVdC1utyZET1PE8cuUg"

public_url = ngrok.connect(8787).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, 8787))

if 'client' not in globals():
  !python -m pip install jupyter-server-proxy
  client = Client()

### System settings

In [ ]:
cwd = os.getcwd()

### CUDF für hardware acceleration

In [ ]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py
# import cudf

## Grobes HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

## Neu-Partitionierug
Sodass alle Partitionen etwa die gleiche Größe haben.

In [ ]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

# ddf = dd.read_parquet(input_dir+"*.parquet")
# ddf2 = ddf.repartition(npartitions=50)
# name_function = lambda x: f"data-{x}.parquet"
# ddf2.to_parquet(output_dir, name_function=name_function)

## Author-Inferenz

Ein bisschen die Daten säubern...

In [ ]:
input_dir = cwd+'/data/bzg/unraw2_bzg/'
output_dir = cwd+'/data/bzg/unraw3_bzg/'

In [ ]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].apply(eval, meta=pd.Series(dtype='object'))

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
dask.config.set(scheduler="processes")
ddf["inferred_author"] = None
ddf["inferred_author"] = ddf.body.apply(infer_author, meta=pd.Series(dtype="string"))

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

In [ ]:
auhtor_value_counts.sum().diff()

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf["channels"] = ddf.channels.apply(lambda x: str(x), meta=pd.Series(dtype="string"))

In [ ]:
ddf.inferred_author.value_counts().compute()

In [ ]:
ddf.inferred_author.value_counts().sum().compute()

In [ ]:
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x, meta=pd.Series(dtype=bool))]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)
earnings_ddf.inferred_author.value_counts().head(10)

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

## Vorgehen

Benötigt:
---------
- data_shared/corporation_endings.txt \\
- input_dir = data/unraw3_bzg/ \\

Produziert:
-----------
- data_shared/all_ticker_name_mapper.parquet

--------------

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt. Bzw. generell ist das eine gute Datensäuberungs-Maßnahme. Einerseits verhindern wir,
dass später Aktienkurse einer falschen Aktie zugeordnet wird. Andererseits werden dadurch evtl. auch weniger seriöse Nachrichten herausgefiltert, die nicht
die Kontaktadresszeile des Unternehmens am Ende besitzen, in dem der vollständige Unternehmensname vorkommt.

In [ ]:
input_dir = cwd+'/data/bzg/unraw3_bzg/'
ddf = dd.read_parquet(input_dir)

In [ ]:
all_tickers = ddf.stocks.unique().compute()

In [ ]:
# Tickers sometimes have a dollar sign in front of them.
# This is common practice to indicate that the acronym refers to a stock ticker.
# This results in ~ 4 duplicate entries.
all_tickers = all_tickers.apply(lambda x: x.strip("$"))

In [ ]:
# Some stock tickers are in lowercase.
# Altough yfinance can handle lowercase tickers we uppercase them
# in order to avoid inconsistencies and avoid duplicates.
all_tickers = all_tickers.str.upper()
all_tickers = all_tickers.drop_duplicates()

In [ ]:
colon_tickers = [x for x in all_tickers.values if ":" in x]
print(f"Around {len(colon_tickers)} stock tickers are from foreign exchanges. \n"
      f"The list of foreign exchanges is:")
set([x.split(":")[0] for x in colon_tickers])

In [ ]:
# We remove these foreign exchanges:
all_tickers = all_tickers[all_tickers.apply(lambda x: ":" not in x)]

In [ ]:
# Do Whitespaces exit and prevent yfinance from finding company names?
# Only one company, and its not even a ticker but the company name
all_tickers[all_tickers.apply(lambda x: " " in x)]

### Full-Name-Discovery

In [ ]:
company_names = all_tickers.map(lambda x: yahoo_get_wrapper(x))

In [ ]:
all_mapper = pd.concat([all_tickers, company_names], axis=1)

In [ ]:
all_mapper.columns = ["ticker", "company_name"]

In [ ]:
print(f"For {all_mapper[all_mapper.isna().any(axis=1)].shape[0]} tickers, yfinance had no entry, or at least not entry for the `longName`")
# E.g. AIS, PTNR, BSI, LUFK, BFRM. Can't find these stocks on guidants as well and
# Some are not headquartered in the US.

In [ ]:
all_mapper = all_mapper.dropna()

In [ ]:
# Multiple tickers for the same company exist for ~556/2 companies
vcs = all_mapper.company_name.value_counts()
vcs = vcs[vcs >= 2]
all_mapper[all_mapper.company_name.isin(vcs.index)].sort_values("company_name").shape[0]

Es ist nicht leicht zu sagen, welchen von den Tickern wir bevorzugen sollten. Abgleichen mit den Aktientickern des Kursdatensatzes notwendig, um zu sehen, ob überhaupt nur ein Ticker übereinstimmt. Wenn es für beide Ticker eine Kurszeitreihe gibt, dann sollten wir die nehmen, die ein höheres historisches Volumen hat. Dies ist allerdings etwas, was wir später machen und nicht jetzt. Hier wollen wir zunächst nur die Nachrichten verarbeiten, weswegen wir nur die NaN-Unternehmen rausnehmen und den Rest - *ohne Ticker-Filtering* - weiterverarbeiten.

In [ ]:
mapper = all_mapper

In [ ]:
mapper.columns = ["ticker", "company_names"]
mapper.set_index("ticker", inplace=True)

In [ ]:
company_endings = pd.read_table("data_shared/corporation_endings.txt").iloc[:, 0]
# Apply `get_company_abbreviation` twice in order to get rid of Enterprise, Ltd.
# Otherwise , Ltd. remains. If no acronym, name stays as is.
mapper["short_name"] = mapper.company_names.apply(lambda x: get_company_abbreviation(x, company_endings=company_endings))

In [ ]:
mapper = mapper.applymap(lambda x: x.strip(" "))

In [ ]:
mapper.to_parquet(cwd + "/data/bzg_shared/ticker_name_mapper.parquet")

In [ ]:
mapper = pd.read_parquet(cwd + "/data/bzg_shared/ticker_name_mapper.parquet")

In [ ]:
filt_ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]

In [ ]:
n = ddf.map_partitions(lambda x: x.shape[0]).compute()
print(f"Es verbleiben {n.sum()} Nachrichten, für die wir den Ticker zu einem Firmennamen auflösen können.")

In [ ]:
n = filt_ddf.map_partitions(lambda x: x.shape[0]).compute()
n.sum()

In [ ]:
ddf = filt_ddf

In [ ]:
ddf["company_name"] = ddf.stocks.map(lambda x: mapper.company_names.loc[x], meta=pd.Series(dtype="string"))
ddf["short_name"] = ddf.stocks.map(lambda x: mapper.short_name.loc[x], meta=pd.Series(dtype="string"))

### Duplikate Entfernen

In [ ]:
n = ddf.map_partitions(lambda x: x.shape[0]).compute().sum()
n

In [ ]:
ddf = ddf.map_partitions(lambda df: df.drop_duplicates())

In [ ]:
n = ddf.map_partitions(lambda x: x.shape[0]).compute().sum()
n

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/bzg/latest/', name_function=name_function)

### Firmennamen-Nachrichtenkörper-Verifikation

In [ ]:
ddf = dd.read_parquet(cwd+'/data/bzg/latest/')

In [ ]:
ddf.dtypes

In [ ]:
# If short_name doesn't occurr in the title or in the body, then article seems to be faulty
mask = ddf.apply(lambda x:
                 bool(re.search(x["short_name"],
                                x.title + x["body"].replace("( )*\n( )*", " "),
                                re.IGNORECASE)),
                 axis=1,
                 meta=pd.Series(dtype=bool))

In [ ]:
print(f"Around {len(ddf.stocks.unique())} stocks before filtering")
print(f"Around {len(ddf[mask].stocks.unique())} stocks after filtering")

In [ ]:
# Filter out faulty news
ddf = ddf[mask]

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/bzg/latest2/', name_function=name_function)

### Timedeltas zwischen Nachrichtenmeldungen



Wir sehen, dass einige Nachrichten dupliziert vorkommen, d.h. mit einem Timedelta von 0 und mit derselben Überschrift etc. diese gilt es zu eliminieren.

#### Convert ddf to pd.DataFrame

In [ ]:
ddf = dd.read_parquet(cwd+'/data/bzg/latest2/')

In [ ]:
ddf = ddf.compute()
ddf = ddf.sort_values("time")

In [ ]:
tmp = ddf[["time", "stocks"]]

In [ ]:
#### Adding timedeltas to the data frame
news_timedeltas = tmp.groupby("stocks").transform(lambda x: x.diff())

In [ ]:
# ~3 minutes evaluates to true
# (news_timedeltas.index == ddf.index).all()

In [ ]:
ddf.loc[:, "timedelta"] = news_timedeltas.time.fillna(pd.Timedelta(days=100))

In [ ]:
news_timedeltas = ddf.timedelta

In [ ]:
news_timedeltas.iloc[0].components

In [ ]:
same_day_timedeltas = news_timedeltas.apply(lambda x: x.components.days == 0)

In [ ]:
(same_day_timedeltas == 0).sum()

In [ ]:
same_hour_timedeltas = news_timedeltas.apply(lambda x: (x.components.days == 0) & \
                                               (x.components.hours == 0))

In [ ]:
print(same_hour_timedeltas.sum())

In [ ]:
same_minute_timedeltas = news_timedeltas.apply(lambda x: (x.components.days == 0) & \
                                               (x.components.hours == 0) & \
                                               (x.components.minutes == 0))

In [ ]:
print(same_minute_timedeltas.sum())

In [ ]:
same_day_ddf = ddf.loc[same_day_timedeltas]

In [ ]:
ddf.stocks.value_counts().describe()

Bis zu 5k Nachrichten pro Firma, z.B. AT&T, was in 13 Jahren ca. einer Nachricht pro Tag entspricht. Wir wollen nicht das eine Firma mit vielen Junk-Nachrichten das Modell dominiert. Wobei das Modell hoffentlich dann auch die Junk-Nachrichten als solche erkennt und ignoriert. Eher wichtig noch einen `staleness`-Faktor, also wie ähnlich die Nachricht zu Vorhergegangenen ist (i.e. Nachrichten desselben Tages oder derselben Woche).

Kategorisieren von Nachrichten (mit Text2Topic, wie Salbrechter?) und eliminieren von Business/Strategic etc.
Im Falle von Text2Topic, versuche Estimates des Unternehmens von Dritten zu unterscheiden.

Wichtig!!! Unterscheide zwischen LERN-Phase und PRODUKTIONS-Phase.
Wir können z.B. CLS-Token in der Produktions-Phase vergleichen, in der Lern-Phase aber noch nicht.

Text2Vec -> Business category evtl. entfernen-> Intrastock variance average

## Nachrichten-Parsing


In [ ]:
ddf = dd.read_parquet(cwd+'/data/bzg/latest2/')

In [ ]:
ticker_name_mapper = pd.read_parquet("data_shared/ticker_name_mapper.parquet")
ticker_name_mapper.shape[0]

In [ ]:
# TODO: Compare to ticker_name_mapper
ticker_name_mapper_reduced = ddf[["stocks", "company_name", "short_name"]].drop_duplicates(keep="first").compute()
ticker_name_mapper_reduced.to_parquet("data_shared/ticker_name_mapper_reduced.parquet")
ticker_name_mapper_reduced.shape[0]

### Beispiel/ Untersuchung

In [ ]:
%%capture
!apt update && \
    python -m nltk.downloader averaged_perceptron_tagger punkt wordnet && \
    DEBIAN_FRONTEND='noninteractive' apt install -y maven

In [ ]:
%%capture
!mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python3 -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec("sutime").origin).parent / "pom.xml")')


In [ ]:
sample_partition = ddf.get_partition(20)
y = sample_partition.head(5)
y.loc[:, "time"] = pd.to_datetime(y.time).dt.tz_convert("UTC")
x = y.iloc[2]
x.body

In [ ]:
res = client.submit(filter_body, row=x, logging=False)

In [ ]:
res.result()

### Anwenden der filter_body-Funktion auf alle Reihen:

In [ ]:
ddf["time"]

In [ ]:
def handle_timezone(x):
    try:
        return x.tz_convert("US/Eastern")
    except Exception as e:
        return x.tz_localize("US/Eastern")

In [ ]:
# Still need to parse time correctly...
# ddf["time"] = ddf["time"].map(lambda x: handle_timezone(pd.to_datetime(x)), meta=pd.Series(dtype="datetime64[ns, US/Eastern]"))

In [ ]:
def check_and_reconnect_drive():
    try:
        # Check if Google Drive is still connected
        os.listdir('/content/drive')
    except:
        # If not, reconnect it
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)

In [ ]:
ddf.get_partition(1)

In [ ]:
for pth in range(15, ddf.npartitions):
  print(f"Start {pth} partition")
  par = ddf.get_partition(pth)
  par["parsed_body"] = par.map_partitions(lambda y: y.apply(lambda x: filter_body(x),
                                                  axis=1),
                                                  meta=pd.Series(dtype="string"))
  par = client.persist(par)
  par.to_parquet(cwd + f"/data/bzg/processed_news", name_function=lambda x: f"data-{pth}.parquet")
  check_and_reconnect_drive()

## Filtern von Newstiteln

## Voranstellen von gefilterten Newstiteln an Nachrichtenkörper

## Analyse der durschnittlichen Tokenlänge